<a href="https://colab.research.google.com/github/Jarde01/ml-tools-and-examples/blob/master/16_class_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Named Entity Recognition with recent and improved version of the Annotated NER Dataset with better features

**Note: It is just a quick demonstration on 'How to use this dataset on Scikit for Named Entity Classification'**
**This is not for analytical concepts and other machine learning illustrations!**

Reading CSV in pandas has many ways. I suggest to use CSV as database to prevent large file handling errors.
One may use the follwing way to read CSV file using Python Pandas. Good Luck

In [0]:
!pip install -U keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [0]:
!pip install --upgrade wandb

In [0]:
!wandb login 2295caae2022fe6a431a755f74d1e9e8605848eb
import wandb
from wandb.keras import WandbCallback
wandb.init(project="16-class-ner")


In [0]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, f1_score

In [0]:
import keras
from keras.preprocessing.text import *
from keras.preprocessing import *
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MultiLabelBinarizer, LabelEncoder

In [0]:
df = pd.read_csv("ner.csv", encoding = "ISO-8859-1", error_bad_lines=False)
df.dropna(inplace=True)

b'Skipping line 281837: expected 25 fields, saw 34\n'


In [0]:
print(len(df['sentence_idx'].unique()))
num_samples = len(df)
num_samples

35178


1050795

In [0]:
filtered_df = filtered_df.filter(['word', 'tag', 'sentence_idx'])
filtered_df.tail()

,word,tag,sentence_idx
253904,the,O,11615
253905,1990s,B-tim,11615
253906,.,O,11615
253907,Ethiopia,B-geo,11616
253908,'s,O,11616


In [0]:
def get_multilabel():
  sentences = []
  tags = []

  sentids = filtered_df['sentence_idx'].unique()

  curr_sent_id = filtered_df['sentence_idx'].iloc[0]
  tokens = []
  tag_tokens = []
  row = None
  for index, row in filtered_df.iterrows():

    sent_id = row['sentence_idx']

    if curr_sent_id == sent_id:
      tokens.append(row['word'])
      tag_tokens.append(row['tag'])
    else:
      curr_sent_id = sent_id
      sentences.append(" ".join(tokens))
      tags.append(tag_tokens)
      tokens = []
      tag_tokens = []

  # get the last sentence
  sentences.append(" ".join(tokens))
  tags.append(tag_tokens)# print(sentences)
  
  x, y, enc, tok = prepare_multilabel(sentences, tags)
  
  return x, y, enc, tok

def prepare_multilabel(sentences, tags):
  # prepare y data
  y = np.array(tags)
  enc = MultiLabelBinarizer()
  y = enc.fit_transform(y)
  print(enc.classes_)
  print(y[0])
  print(y.shape)
  
  # prepare x data
  x = np.array(sentences)
  tok = Tokenizer()
  tok.fit_on_texts(x)
  x = tok.texts_to_sequences(x)
  x = sequence.pad_sequences(x, maxlen=100, padding='post')
  x = np.array(x)
  return x, y, enc, tok


In [0]:
def get_singlelabel():
  words = filtered_df['word'].values
  tags = filtered_df['tag'].values
  
  x, y, (le,enc), tok = prepare_singlelabel(words, tags)
  return x, y, (le, enc), tok
  
def prepare_singlelabel(words, tags):
  
  # prepare x
  x = np.array(words)
  tok = Tokenizer()
  tok.fit_on_texts(x)
  x = tok.texts_to_sequences(x)
  x = sequence.pad_sequences(x, maxlen=1, padding='post')
  x = np.array(x)
    
  # prepare y
  le = LabelEncoder()
  labels = le.fit_transform(tags)
  enc = LabelBinarizer()
  y = enc.fit_transform(labels)
  
  print(enc.classes_)
  print(y[0])
  print(y.shape)
  
  return x, y, le, enc, tok

In [0]:
# x, y = get_multilabel()
x, y, (le, enc), tok = get_singlelabel()

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
(253909, 17)


In [0]:
!pip install dill

In [0]:
import dill
with open("data_utils.pkl", "wb") as f:
  dill.dump((x,y,le,enc,tok), f)

In [0]:
print(len(x))
print(len(y))

253909
253909


In [0]:
 # print(tok.__dict__.keys())
num_words = len(tok.word_index)

In [0]:
num_words

14950

In [0]:
# classes = filtered_df['tag'].unique()
num_classes = y.shape[1]
num_classes

17

Splitting Train and test data

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
print(x_train.shape)
print(y_train.shape)

(177736, 1)
(177736, 17)


In [0]:
def hyperas_data_fn():
  import dill
  with open("data_utils.pkl", "rb") as f:
    x,y,le,enc,tok = dill.load(f)
    return x,y,le,enc,tok 

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, GlobalMaxPooling1D
from keras.optimizers import SGD
from keras import backend as K
  

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

  
def get_model(num_classes, num_words, embed_out=64, dense_1_nodes=64, dropout_1=0.5):
    model = Sequential()
    model.add(Embedding(num_words+1, embed_out, input_length=1, name='embedding_layer'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(dense_1_nodes, activation='relu', name='dense_1'))
    model.add(Dropout(dropout_1))
#     model.add(Dense(num_classes, activation='sigmoid')) # multilabel
    model.add(Dense(num_classes, activation='sigmoid', name='output_layer')) #single label
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
    return model

In [0]:
import random

def run_trials_single():
  run_name = "" # what are you testing with this run
  notes = ""

  #hyperparams
  hyperparams = dict(
    embed_out =random.randint(8,256),
    dense_1_nodes = random.randint(8,256),
    dropout_1=random.uniform(0,0.95),
  )

  config= {
      "epochs": 50,
      "num_classes":num_classes,
      "num_words": num_words,
      "batch_size":batch_size,
      **hyperparams
  }

  model = get_model(num_classes, num_words, **hyperparams)

  wandb.init(project="16-class-ner", name=run_name, config=config, notes=notes)

  history = model.fit(x_train, y_train, batch_size=config['batch_size'], epochs=config['epochs'], 
            validation_split=0.2, callbacks=[WandbCallback()])

  scores = model.evaluate(x_test, y_test)
  wandb.log({'test_'+metric: scores[index] for index, metric in enumerate(model.metrics_names)})
  print("\n*** Finished! ***")

In [0]:
for x in range(0,100):
  run_trials_single()

Train on 142188 samples, validate on 35548 samples
Epoch 1/50
142188/142188 [==============================] - 9s 61us/step - loss: 1.4104 - acc: 0.8390 - f1_m: 0.7810 - precision_m: 0.7365 - recall_m: 0.8951 - val_loss: 0.5011 - val_acc: 0.8652 - val_f1_m: 0.8735 - val_precision_m: 0.8368 - val_recall_m: 0.9137
Epoch 2/50
142188/142188 [==============================] - 6s 44us/step - loss: 0.3394 - acc: 0.9145 - f1_m: 0.9105 - precision_m: 0.8960 - recall_m: 0.9261 - val_loss: 0.3102 - val_acc: 0.9273 - val_f1_m: 0.9229 - val_precision_m: 0.9254 - val_recall_m: 0.9204
Epoch 3/50
142188/142188 [==============================] - 6s 44us/step - loss: 0.2198 - acc: 0.9407 - f1_m: 0.9356 - precision_m: 0.9361 - recall_m: 0.9351 - val_loss: 0.2824 - val_acc: 0.9286 - val_f1_m: 0.9219 - val_precision_m: 0.9300 - val_recall_m: 0.9140
Epoch 4/50
142188/142188 [==============================] - 6s 44us/step - loss: 0.1881 - acc: 0.9440 - f1_m: 0.9382 - precision_m: 0.9374 - recall_m: 0.9389 - 

In [0]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform


def run_trials():
  run_name = ""

  hyperparams = dict(
    embed_out =32,
    dense_1_nodes = 24,
    dropout_1=0.3,
  )

  config= {
      "epochs": 50,
      "num_classes":17,
      "num_words": 14950,
      "batch_size":{{choice([64, 128])}},
      **hyperparams
  }
  
  model = Sequential()
  model.add(Embedding(num_words+1, {{uniform(10,256)}}, input_length=1, name='embedding_layer'))
  model.add(GlobalMaxPooling1D())
  model.add(Dense({{uniform(10,128)}}, activation={{choice(['relu', 'sigmoid'])}}, name='dense_1'))
  model.add(Dropout({{uniform(10,256)}}))
  model.add(Dense(num_classes, activation='sigmoid', name='output_layer'))
  model.compile(loss='categorical_crossentropy', optimizer={{choice(['rmsprop', 'adam', 'sgd'])}}, metrics=['acc',f1_m,precision_m, recall_m])
  
  model = get_model(num_classes, num_words, **hyperparams)

  wandb.init(project="16-class-ner", name=run_name, config=config, notes=notes)

  history = model.fit(x_train, y_train, batch_size=config['batch_size'], epochs=config['epochs'], validation_split=0.2, callbacks=[WandbCallback()])

  scores = model.evaluate(x_test, y_test)
  wandb.log({'test_'+metric: scores[index] for index, metric in enumerate(model.metrics_names)})
  print("\n*** Finished! ***")

  validation_acc = np.amax(history.history['val_acc']) 
  print('Best validation acc of epoch:', validation_acc)
  return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

best_run, best_model = optim.minimize(model=run_trials, data=hyperas_data_fn, algo=tpe.suggest, max_evals=100, trials=Trials(),notebook_name='16_class_ner(1)')

>>> Imports:
#coding=utf-8

try:
    import wandb
except:
    pass

try:
    from wandb.keras import WandbCallback
except:
    pass

try:
    import json
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from sklearn.feature_extraction import DictVectorizer
except:
    pass

try:
    from sklearn.linear_model import Perceptron
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.metrics import precision_recall_fscore_support, f1_score
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.preprocessing.text import *
except:
    pass

try:
    from keras.preprocessing import *
except:
    pass

try:
    from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MultiLabelBinarizer, LabelEncoder
except:
    pass

try:
    import dill
except:
    pass

try:
    import dill
except:
    pass

try:
    import keras
except:
    p

IndexError: ignored

In [0]:
!pip install hyperas

     |████████████████████████████████| 337kB 7.4MB/s 
ERROR: ipython 5.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.4, but you'll have prompt-toolkit 2.0.9 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.16
    Uninstalling prompt-toolkit-1.0.16:
      Successfully uninstalled prompt-toolkit-1.0.16


In [0]:
test_input = tok.texts_to_sequences(['prime'])
print(test_input)
pred = model.predict(test_input)
np.argmax(pred)

enc.__dict__

[[99]]


{'_cached_dict': None,
 'classes': None,
 'classes_': array(['-', 'B', 'I', 'O', 'a', 'e', 'g', 'i', 'm', 'n', 'o', 'p', 'r',
        't', 'v'], dtype=object),
 'sparse_output': False}